In [1]:
import pandas as pd
import numpy as np
#from pycaret.regression import *
from pycaret.clustering import *
import warnings
#import seaborn as sns
#import matplotlib.pyplot as plt

import os
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

warnings.filterwarnings("ignore")

In [2]:
from pycaret.utils import version
version()

'2.3.10'

In [3]:
data = []

for diri in os.listdir('can_bus_audi/'):
    for i in os.listdir('can_bus_audi/'+diri+'/bus'):
        data.append(pd.read_json('can_bus_audi/'+diri+'/bus/'+i))
        #with open('can_bus_audi/'+diri+'/bus/'+i,'r') as f:
            #print('can_bus_audi/'+diri+'/bus/'+i)
            #pd.([data, pd.json_normalize(json.loads(f.read()))])
            
df_data = pd.concat(data).drop('unit')

In [4]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, values to values
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   acceleration_x                   3 non-null      object
 1   acceleration_y                   3 non-null      object
 2   acceleration_z                   3 non-null      object
 3   accelerator_pedal                3 non-null      object
 4   accelerator_pedal_gradient_sign  3 non-null      object
 5   angular_velocity_omega_x         3 non-null      object
 6   angular_velocity_omega_y         3 non-null      object
 7   angular_velocity_omega_z         3 non-null      object
 8   brake_pressure                   3 non-null      object
 9   distance_pulse_front_left        3 non-null      object
 10  distance_pulse_front_right       3 non-null      object
 11  distance_pulse_rear_left         3 non-null      object
 12  distance_pulse_rear_right        3 

In [5]:
df_data.isna().sum()

acceleration_x                     0
acceleration_y                     0
acceleration_z                     0
accelerator_pedal                  0
accelerator_pedal_gradient_sign    0
angular_velocity_omega_x           0
angular_velocity_omega_y           0
angular_velocity_omega_z           0
brake_pressure                     0
distance_pulse_front_left          0
distance_pulse_front_right         0
distance_pulse_rear_left           0
distance_pulse_rear_right          0
latitude_degree                    0
latitude_direction                 0
longitude_degree                   0
longitude_direction                0
pitch_angle                        0
roll_angle                         0
steering_angle_calculated          0
steering_angle_calculated_sign     0
vehicle_speed                      0
dtype: int64

In [6]:
df_data.head()

,acceleration_x,acceleration_y,acceleration_z,accelerator_pedal,accelerator_pedal_gradient_sign,angular_velocity_omega_x,angular_velocity_omega_y,angular_velocity_omega_z,brake_pressure,distance_pulse_front_left,distance_pulse_front_right,distance_pulse_rear_left,distance_pulse_rear_right,latitude_degree,latitude_direction,longitude_degree,longitude_direction,pitch_angle,roll_angle,steering_angle_calculated,steering_angle_calculated_sign,vehicle_speed
values,"[[1554121593489120, 0.319999999999936], [15541...","[[1554121593489120, 0.220000000000027], [15541...","[[1554121593489128, 10.0], [1554121593494102, ...","[[1554121593495095, 22.8], [1554121593505094, ...","[[1554121593495095, 0], [1554121593505094, 0],...","[[1554121593489128, -0.40000000000003405], [15...","[[1554121593489128, 0.49000000000000904], [155...","[[1554121593489120, -0.32999999999998403], [15...","[[1554121593490045, 0.0], [1554121593500040, 0...","[[1554121593505032, 391], [1554121593525035, 4...","[[1554121593505032, 97], [1554121593525035, 10...","[[1554121593505032, 636], [1554121593525035, 6...","[[1554121593505032, 759], [1554121593525035, 7...","[[1554121593609081, 48.771678], [1554121593769...","[[1554121593609081, 0], [1554121593769081, 0],...","[[1554121593609081, 11.418433], [1554121593769...","[[1554121593609081, 0], [1554121593769081, 0],...","[[1554121593494102, 0.799999999999997], [15541...","[[1554121593494102, 1.025000000000005], [15541...","[[1554121593494041, 0.45], [1554121593504050, ...","[[1554121593494041, 1], [1554121593504050, 1],...","[[1554121593505032, 41.47], [1554121593525035,..."
values,"[[1554115464693201, -0.5], [1554115464698160, ...","[[1554115464693201, 0.319999999999936], [15541...","[[1554115464693212, 9.819999999999936], [15541...","[[1554115464699170, 0.0], [1554115464709186, 0...","[[1554115464699170, 0], [1554115464709186, 0],...","[[1554115464693212, 0.39999999999997704], [155...","[[1554115464693212, -0.519999999999981], [1554...","[[1554115464693201, -0.13999999999998602], [15...","[[1554115464694125, 6.299999999999997], [15541...","[[1554115464699122, 88], [1554115464719125, 89...","[[1554115464699122, 897], [1554115464719125, 8...","[[1554115464699122, 167], [1554115464719125, 1...","[[1554115464699122, 546], [1554115464719125, 5...","[[1554115464763134, 48.142837], [1554115464923...","[[1554115464763134, 0], [1554115464923127, 0],...","[[1554115464763134, 11.576722], [1554115464923...","[[1554115464763134, 0], [1554115464923127, 0],...","[[1554115464699097, -0.125], [1554115464709106...","[[1554115464699097, 2.450000000000003], [15541...","[[1554115464698116, 2.4], [1554115464708106, 2...","[[1554115464698116, 1], [1554115464708106, 1],...","[[1554115464699122, 6.54], [1554115464719125, ..."
values,"[[1533906414544846, 4.319999999999936], [15339...","[[1533906414544846, 0.419999999999959], [15339...","[[1533906414544858, 9.580000000000041], [15339...","[[1533906414544966, 38.8], [1533906414554935, ...","[[1533906414544966, 0], [1533906414554935, 0],...","[[1533906414544858, -0.550000000000011], [1533...","[[1533906414544858, -1.009999999999991], [1533...","[[1533906414544846, 3.370000000000004], [15339...","[[1533906414549891, 0.0], [1533906414559893, 0...","[[1533906414554882, 179], [1533906414574879, 1...","[[1533906414554882, 84], [1533906414574879, 86...","[[1533906414554882, 658], [1533906414574879, 6...","[[1533906414554882, 464], [1533906414574879, 4...","[[1533906414558865, 48.788047], [1533906414718...","[[1533906414558865, 0], [1533906414718868, 0],...","[[1533906414558865, 11.382514], [1533906414718...","[[1533906414558865, 0], [1533906414718868, 0],...","[[1533906414544858, -1.075000000000002], [1533...","[[1533906414544858, 1.700000000000002], [15339...","[[1533906414543862, 52.35], [1533906414553856,...","[[1533906414543862, 0], [1533906414553856, 0],...","[[1533906414554882, 10.36], [1533906414574879,..."


In [7]:
clean_data = []
for i in df_data.columns:
    clean_data.append(np.asanyarray(df_data[i].values.all()).flatten().reshape(-1,2)[:,1])

In [8]:
df = pd.DataFrame(data=clean_data).transpose()

In [9]:
df = pd.DataFrame(data=df.values, columns=df_data.columns)

In [10]:
df.head()

,acceleration_x,acceleration_y,acceleration_z,accelerator_pedal,accelerator_pedal_gradient_sign,angular_velocity_omega_x,angular_velocity_omega_y,angular_velocity_omega_z,brake_pressure,distance_pulse_front_left,distance_pulse_front_right,distance_pulse_rear_left,distance_pulse_rear_right,latitude_degree,latitude_direction,longitude_degree,longitude_direction,pitch_angle,roll_angle,steering_angle_calculated,steering_angle_calculated_sign,vehicle_speed
0,4.32,0.42,9.58,38.8,0.0,-0.55,-1.01,3.37,0.0,179.0,84.0,658.0,464.0,48.788047,0.0,11.382514,0.0,-1.075,1.700,52.35,0.0,10.36
1,4.34,0.40,9.58,38.8,0.0,-0.51,-1.10,3.31,0.0,182.0,86.0,660.0,466.0,48.788047,0.0,11.382514,0.0,-1.075,1.700,51.30,0.0,10.66
2,4.26,0.42,9.70,38.8,0.0,-0.46,-0.83,3.25,0.0,184.0,89.0,663.0,469.0,48.788047,0.0,11.382514,0.0,-1.100,1.700,50.10,0.0,10.98
3,4.30,0.48,9.68,38.8,0.0,-0.70,-0.75,3.21,0.0,187.0,91.0,665.0,472.0,48.788047,0.0,11.382514,0.0,-1.100,1.700,49.05,0.0,11.28
4,4.36,0.44,9.76,38.8,0.0,-0.73,-0.85,3.26,0.0,190.0,94.0,668.0,474.0,48.788047,0.0,11.382496,0.0,-1.100,1.675,48.00,0.0,11.58


In [11]:
exp = setup(data=df,
            normalize=True,
            normalize_method='minmax',
            session_id=42
           )

,Description,Value
0,session_id,42
1,Original Data,"(105153, 22)"
2,Missing Values,True
3,Numeric Features,18
4,Categorical Features,4
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(105153, 24)"
9,CPU Jobs,-1


In [12]:
kmeans = create_model('kmeans',num_clusters = 4)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.3282,65305.0597,0.9537,0,0,0


In [ ]:

plot_model(kmeans,plot='tsne')

IntProgress(value=0, description='Processing: ', max=5)

In [ ]:
np.inf